# **ETL - POBLACION**

In [1]:
# [Configuración] Importación de librerías
import pandas as pd
import numpy as np

depart = pd.read_csv("../ARCHIVOS/Poblacion/poblacion_departamental_2018-2050.csv", encoding='latin-1', low_memory=False)
munici = pd.read_csv("../ARCHIVOS/Poblacion/poblacion_municipal_2018_2042.csv", encoding='utf-8', low_memory=False)
munici = munici[munici['DP'] == 8]

depart = depart.drop(columns=['DP'])
munici = munici.drop(columns=['DP', 'DPMP', 'DPNOM'])
depart = depart.rename(columns={'DPNOM': 'DEPARTAMENTO', 'TOTAL': 'POBLACIÓN'})
depart['POBLACIÓN'] = (
    depart['POBLACIÓN']
    .astype(str)
    .str.replace(',', '', regex=True)
    .astype(int)
)

depart['DEPARTAMENTO'] = depart['DEPARTAMENTO'].astype(str).str.upper()
munici['MUNICIPIO'] = munici['MUNICIPIO'].astype(str).str.upper()

depart = depart.sort_values(by=['AÑO', 'DEPARTAMENTO'], ascending=[True, True])
munici = munici.sort_values(by=['AÑO', 'MUNICIPIO'], ascending=[True, True])

def restrict(data):
    data = data[(data["AÑO"] >= 2017) & (data["AÑO"] <= 2023)]
    data = data[data["ÁREA GEOGRÁFICA"] == "Total"]
    data = data.drop(columns=['ÁREA GEOGRÁFICA'])
    return data

depart = restrict(depart)
munici = restrict(munici)

In [2]:
def pivot(data, unidad):
    tabla_wide = (
        data.pivot_table(
            index=unidad,       # lo que queda como fila
            columns="AÑO",           # cada año será una columna
            values="POBLACIÓN",      # lo que se distribuye
            aggfunc="sum",           # suma (por si hay más de un registro)
            fill_value=0             # si falta valor, poner 0
        )
        .reset_index()
    )

    # Agregar columna con total por municipio
    tabla_wide["TOTAL"] = tabla_wide.drop(columns=unidad).sum(axis=1)

    display(tabla_wide)
    return tabla_wide

In [3]:
pob_nac = pivot(depart, 'DEPARTAMENTO')


AÑO,DEPARTAMENTO,2018,2019,2020,2021,2022,2023,TOTAL
0,AMAZONAS,76573,78065,79583,80865,81996,83087,480169
1,ANTIOQUIA,6407149,6518070,6642072,6721140,6777249,6826125,39891805
2,ARAUCA,262177,266031,270495,273137,274806,276240,1622886
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",61284,61679,62127,62573,62801,63007,373471
4,ATLÁNTICO,2535506,2608908,2693388,2741296,2774009,2805401,16158508
5,"BOGOTÁ, D.C.",7391056,7533202,7717564,7804920,7849206,7883928,46179876
6,BOLÍVAR,2070083,2108412,2151461,2177352,2195275,2210979,12913562
7,BOYACÁ,1218755,1233430,1248977,1260227,1268367,1275286,7505042
8,CALDAS,998249,1011586,1025648,1035206,1041232,1044570,6156491
9,CAQUETÁ,402085,406111,410567,414947,418724,422271,2474705


In [4]:
pob_mun = pivot(munici, 'MUNICIPIO')

AÑO,MUNICIPIO,2018,2019,2020,2021,2022,2023,TOTAL
0,BARANOA,62371,64550,66927,68800,70464,72072,405184
1,BARRANQUILLA,1206322,1230835,1260910,1271797,1274895,1277216,7521975
2,CAMPO DE LA CRUZ,22876,23641,24454,25124,25669,26217,147981
3,CANDELARIA,15977,16579,17240,17758,18197,18647,104398
4,GALAPA,60724,62945,65478,66754,67560,68313,391774
5,JUAN DE ACOSTA,21005,21596,22270,22748,23149,23545,134313
6,LURUACO,28173,29047,30001,30713,31315,31844,181093
7,MALAMBO,128199,133000,138416,141679,144155,146583,832032
8,MANATÍ,19803,20521,21325,21908,22405,22882,128844
9,PALMAR DE VARELA,28928,30007,31160,32090,32941,33778,188904


In [7]:
pob_nac.to_csv("../DATA/poblacion_nacional.csv", encoding='latin_1', index=False)
pob_mun.to_csv("../DATA/poblacion_depart.csv", encoding='latin_1', index=False)